In [27]:

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
import warnings
warnings.filterwarnings("ignore")
import os

In [29]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [30]:

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB


In [31]:
train.columns

Index(['id', 'Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
       'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency', 'Personality'],
      dtype='object')

In [32]:
train.describe()

,id,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,18524.000000,17334.000000,17344.000000,17058.000000,17470.000000,17260.000000
mean,9261.500000,3.137764,5.265106,4.044319,7.996737,4.982097
std,5347.562529,3.003786,2.753359,2.062580,4.223484,2.879139
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4630.750000,1.000000,3.000000,3.000000,5.000000,3.000000
50%,9261.500000,2.000000,5.000000,4.000000,8.000000,5.000000
75%,13892.250000,4.000000,8.000000,6.000000,12.000000,7.000000
max,18523.000000,11.000000,10.000000,7.000000,15.000000,10.000000


In [33]:
train.isnull().sum()

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [34]:
test.isnull().sum()

id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64

In [35]:
import sklearn.impute
dir(sklearn.impute)
help(sklearn.impute.MissingIndicator)

Help on class MissingIndicator in module sklearn.impute._base:

class MissingIndicator(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  MissingIndicator(*, missing_values=nan, features='missing-only', sparse='auto', error_on_new=True)
 |
 |  Binary indicators for missing values.
 |
 |  Note that this component typically should not be used in a vanilla
 |  :class:`~sklearn.pipeline.Pipeline` consisting of transformers and a
 |  classifier, but rather could be added using a
 |  :class:`~sklearn.pipeline.FeatureUnion` or
 |  :class:`~sklearn.compose.ColumnTransformer`.
 |
 |  Read more in the :ref:`User Guide <impute>`.
 |
 |  .. versionadded:: 0.20
 |
 |  Parameters
 |  ----------
 |  missing_values : int, float, str, np.nan or None, default=np.nan
 |      The placeholder for the missing values. All occurrences of
 |      `missing_values` will be imputed. For pandas' dataframes with
 |      nullable integer dtypes with missing values, `missing_values`
 |      should be set 

### Handling missing values

In [36]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer, SimpleImputer,MissingIndicator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
# Identify features
train_cat_features = train.select_dtypes(include=['object']).columns.tolist()
train_num_features = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
test_cat_features = test.select_dtypes(include=['object']).columns.tolist()
test_num_features = test.select_dtypes(include=['int64', 'float64']).columns.tolist()


mis = MissingIndicator()
k = mis.fit(train[train_num_features])

# Fill numerical missing values with KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)
train_num_imputed = knn_imputer.fit_transform(train[train_num_features])
test_num_imputed = knn_imputer.transform(test[test_num_features])   

train_num_imputed = pd.DataFrame(train_num_imputed, columns=train_num_features)
test_num_imputed = pd.DataFrame(test_num_imputed, columns=test_num_features)
train_num_imputed


# Fill categorical missing values with mode



,id,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
0,0.0,0.0,6.0,4.0,15.0,5.0
1,1.0,1.0,7.0,3.0,10.0,8.0
2,2.0,6.0,1.0,0.0,3.0,0.0
3,3.0,3.0,7.0,3.0,11.0,5.0
4,4.0,1.0,4.0,4.0,13.0,5.0
...,...,...,...,...,...,...
18519,18519.0,3.0,7.0,3.0,9.0,7.0
18520,18520.0,1.0,6.0,7.0,6.0,5.0
18521,18521.0,7.0,1.0,1.0,1.0,5.6
18522,18522.0,3.4,1.0,0.0,5.0,2.0


In [37]:
train_num_imputed.isnull().sum()

id                         0
Time_spent_Alone           0
Social_event_attendance    0
Going_outside              0
Friends_circle_size        0
Post_frequency             0
dtype: int64

In [38]:
for col in train_num_features:
    print(train[col].isnull().sum())

0
1190
1180
1466
1054
1264


In [39]:
print(train.isnull().sum())
test.isnull().sum()

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64


id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64

### Encoding Categorical Features (Label Encoding)

In [40]:
# Encode categorical features (train and test together)
for col in cat_features:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

NameError: name 'cat_features' is not defined

### Encode Target & Split

In [ ]:
# Encode target
target_le = LabelEncoder()
train['Personality'] = target_le.fit_transform(train['Personality'])  # Introvert/Extrovert

X = train.drop(['id', 'Personality'], axis=1)
y = train['Personality']
X_test = test.drop(['id'], axis=1)

## Random forest

- An ensemble learning method using multiple decision trees.
- Achieved ~0.9655 validation accuracy.

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# model = RandomForestClassifier(n_estimators=200, random_state=42)
# model.fit(X_train, y_train)

# # Validation accuracy
# y_pred = model.predict(X_val)
# acc = accuracy_score(y_val, y_pred)
# print(f"Validation Accuracy: {acc:.4f}")

## XGBoost

- A gradient boosting framework optimized for speed and performance.
- Achieved up to ~0.9679 accuracy using boosting strategy.

In [ ]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# X = train.drop(columns=['id', 'Personality'])
# y = train['Personality']
# X_test = test.drop(columns=['id'])

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# model = xgb.XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, use_label_encoder=False, eval_metric='logloss')
# model.fit(X_train, y_train)

# y_pred = model.predict(X_val)
# acc = accuracy_score(y_val, y_pred)
# print(f"XGBoost Accuracy: {acc:.4f}")

## CatBoost

- Gradient boosting library optimized for categorical features.
-  Achieved up to ~0.9690 accuracy using boosting strategy.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# Mapping (You MUST define this based on your dataset)
# Example: 0 -> Introvert, 1 -> Extrovert
label_map = {0: "Introvert", 1: "Extrovert"}

# Handle missing values
for df in [train, test]:
    df['Stage_fear'] = df['Stage_fear'].fillna(df['Stage_fear'].mode()[0])
    df['Drained_after_socializing'] = df['Drained_after_socializing'].fillna(df['Drained_after_socializing'].mode()[0])
    num_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
    for col in num_cols:
        df[col] = df[col].fillna(df[col].mean())

# Categorical features
cat_cols = ['Stage_fear', 'Drained_after_socializing']

# Features and Target
X = train.drop(columns=['id', 'Personality'])
y = train['Personality']
X_test = test.drop(columns=['id'])
test_ids = test['id']

# Split for validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Train CatBoost
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    cat_features=cat_cols,
    eval_metric="Accuracy",
    early_stopping_rounds=100,
    verbose=False
)

model.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)

# Evaluate accuracy
val_preds = model.predict(X_valid)
val_acc = accuracy_score(y_valid, val_preds)
#print(f"✅ CatBoost Validation Accuracy: {val_acc:.4f}")

# Predict on test set
test_preds_num = model.predict(X_test)

# Map numeric predictions to original labels
test_preds_label = [label_map[i] for i in test_preds_num]

# Create submission
submission = pd.DataFrame({
    "id": test_ids,
    "Personality": test_preds_label
})

submission.to_csv("submission.csv", index=False)
# print("\n✅ Submission saved as 'submission.csv'")
print(submission.head())

      id Personality
0  18524   Introvert
1  18525   Extrovert
2  18526   Introvert
3  18527   Introvert
4  18528   Extrovert
